In [83]:
import requests
import datetime

# Today's date
today = datetime.datetime.now()

competition_ids = list()

# Generate the link for all WCs and get all comp ids
for season in range(int(str(today.year)[-2:]) + 1):
    
    # Generating URLs for world cups since 2000 until this year
    url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?type=ranking&cat=ICC_MB&cup='
    url = url + str(season).rjust(2,'0') + '_WC'
    overall_json = requests.get(url).json()
    
    # Part of the API that has the comp IDs needed
    season_competition_ids = list(overall_json['route_names'].keys())
    
    # Getting all comp ids
    for comp in season_competition_ids:
        competition_ids.append(int(comp))

In [84]:
url

'http://egw.ifsc-climbing.org/egw/ranking/json.php?type=ranking&cat=ICC_MB&cup=19_WC'

In [80]:
for competition_id in competition_ids:
    url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
    competition_id = competition_ids[-15]
    category = 5
    route = 3
    
    # Adding competition, category and route to url
    url = url + 'comp=' + str(competition_id) + '&cat=' + str(category) + '&route=' + str(route)
    comp_json = requests.get(url).json()
    

In [85]:
url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
competition_id = competition_ids[-15]
category = 5
route = 3

# competitions = {competition_id:}

# Adding competition, category and route to url
url = url + 'comp=' + str(competition) + '&cat=' + str(category) + '&route=' + str(route)
url
comp_json = requests.get(url).json()
comp_json

{'WetId': '6158',
 'GrpId': '5',
 'route_order': '3',
 'route_name': 'Final W O M E N bouldering',
 'route_judge': 'Matevz Gradisek, David Mora Garcia',
 'route_iso_open': '16.30',
 'route_iso_close': '17.30',
 'route_start': '18.30',
 'route_num_problems': '4',
 'route_climbing_time': '4 min.',
 'discipline': 'boulder',
 'comp_name': 'IFSC Climbing Worldcup (B) - Meiringen (SUI) 2017 ',
 'comp_date': '2017-04-07',
 'display_athlete': 'nation',
 'route_names': {'-1': 'General result',
  '0': '1. Qualification',
  '1': '2. Qualification',
  '2': 'Semi-Final',
  '3': 'Final'},
 'category_offical': True,
 'current': [],
 'participants': [{'PerId': '8247',
   'acl': '63',
   'birthyear': '1993',
   'boulder1': 't2 b2',
   'boulder2': 't1 b1',
   'boulder3': 't3 b3',
   'boulder4': 't4 b2',
   'checked': True,
   'city': 'Sheffield',
   'fed_id': '7',
   'fed_url': 'http://www.thebmc.co.uk/',
   'federation': 'British Mountaineering Council',
   'firstname': 'Shauna',
   'lastname': 'Coxsey

In [86]:
url

'http://egw.ifsc-climbing.org/egw/ranking/json.php?comp=6158&cat=5&route=3'

In [ ]:
class Competition:
    def __init__(self, name, id, date, discipline, comp_start, category, iso_open, iso_close)